# Preparation

## Import libraries

In [1]:
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import pytz
from datetime import datetime
import urllib.request, json, os, os.path, csv
from pprint import pprint

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import TimeoutException

## Define function

In [2]:
def zonasi(row):
    if int(row['total_positif']) > 0:
        return('Merah')
    elif int(row['total_pdp']) > 0:
        return('Oranye')
    elif int(row['total_odp']) > 0:
        return('Hijau')
    else: return('Biru')

## Setup chrome driver

In [3]:
driver_path = 'D:/Data Science/Python/chromedriver.exe'

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

tz = pytz.timezone('Asia/Jakarta')
default_date_update = datetime.now(tz).strftime("%d-%b-%Y")

format_header = ['scrape_date', 'date_update', 'waktu', 'provinsi', 'kabkot', 'kecamatan', 
                 'kelurahan', 'alamat', 'total_odp', 'total_pdp', 'total_positif', 'total_otg',
                 'odr_total', 'total_pp', 'total_ppdt', 'source_link', 'zona', 'Ket']

# Web Scrapping

In [4]:
samarinda_dataset = []

now = datetime.now(tz)

# Data Global
driver = webdriver.Chrome(driver_path, options=options)
url = 'https://corona.samarindakota.go.id/'
driver.get(url)
time.sleep(5)
new_corpus = {'scrape_date':now.strftime("%d-%b-%Y"),
              'date_update':datetime.strptime(re.findall('\d+ \w+ \d+', driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/section[5]/div/div[1]/p').text.replace('Juli', 'Jul'))[0], '%d %b %Y').strftime('%d-%b-%Y'),
              'waktu':now.strftime("%H:%M"),
              'provinsi':'Kalimantan Timur',
              'kabkot':'Kota Samarinda',
              'total_odp':int(re.findall("\d+", driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/section[5]/div/div[2]/div/div/div/div[2]/p').text)[0]),
              'total_pdp':int(re.findall("\d+", driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/section[5]/div/div[3]/div/div/div/div[2]/p').text)[0]),
              'total_positif':int(re.findall("\d+", driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/section[5]/div/div[4]/div/div/div/div[2]/div/div[2]/p[1]').text)[0]),
              'source_link':url,
              'Ket':'Total kasus seluruh kecamatan.'}
if new_corpus not in samarinda_dataset: samarinda_dataset.append(new_corpus)
else: print('Identical corpus already exist in dataset.')
driver.quit()

df = pd.DataFrame(columns=format_header)

for data in samarinda_dataset:
    df = df.append(data, ignore_index=True)

num_nan = ['total_odp', 'total_pdp', 'total_positif', 'total_otg', 
           'odr_total', 'total_pp', 'total_ppdt']
df[num_nan] = df[num_nan].fillna(0)
str_nan = ['kecamatan', 'kelurahan', 'alamat', 'Ket']
df[str_nan] = df[str_nan].fillna('')

df['zona'] = df.apply(lambda row: zonasi(row), axis=1)

df.to_excel('{0}_Kecamatan_Samarinda.xlsx' .format(datetime.now(tz).strftime("%Y%m%d")), index=False)

print('Shape of df:', df.shape)
df.head()

Shape of df: (1, 18)


,scrape_date,date_update,waktu,provinsi,kabkot,kecamatan,kelurahan,alamat,total_odp,total_pdp,total_positif,total_otg,odr_total,total_pp,total_ppdt,source_link,zona,Ket
0,27-Jul-2020,18-Jul-2020,18:47,Kalimantan Timur,Kota Samarinda,,,,2717,73,71,0,0,0,0,https://corona.samarindakota.go.id/,Merah,Total kasus seluruh kecamatan.
